In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from energyemissionsregio.config import SHP_PATH

In [ ]:
nuts2 = gpd.read_file(os.path.join(SHP_PATH, "NUTS2.shp"))
nuts2 = nuts2[nuts2["code"].str.startswith(("ES", "DE"))]

In [ ]:
disagg_rail = pd.read_csv(r"C:\Users\s.patil\OneDrive - Forschungszentrum Jülich GmbH\Documents\code\EnergyEmissionsRegio\data\disaggregated_data\GHG\ghg_emissions_from_fc_in_rail_transport.csv")
disagg_road  = pd.read_csv(r"C:\Users\s.patil\OneDrive - Forschungszentrum Jülich GmbH\Documents\code\EnergyEmissionsRegio\data\disaggregated_data\GHG\ghg_emissions_from_fc_in_road_transport_using_cars.csv")
disagg_road_freight  = pd.read_csv(r"C:\Users\s.patil\OneDrive - Forschungszentrum Jülich GmbH\Documents\code\EnergyEmissionsRegio\data\disaggregated_data\GHG\ghg_emissions_from_fc_in_road_freight_transport.csv")
disagg_motor_cycles  = pd.read_csv(r"C:\Users\s.patil\OneDrive - Forschungszentrum Jülich GmbH\Documents\code\EnergyEmissionsRegio\data\disaggregated_data\GHG\ghg_emissions_from_fc_in_road_transport_using_motorcycles.csv")

In [ ]:
disagg_merge_1 = pd.merge(disagg_rail, disagg_road, on="region_code")
disagg_merge_1["value"] = disagg_merge_1["value_x"] + disagg_merge_1["value_y"]
disagg_merge_1.drop(columns=["value_x", "value_y"], inplace=True)

disagg_merge_2 = pd.merge(disagg_road_freight, disagg_motor_cycles, on="region_code")
disagg_merge_2["value"] = disagg_merge_2["value_x"] + disagg_merge_2["value_y"]
disagg_merge_2.drop(columns=["value_x", "value_y"], inplace=True)

disagg_merge = pd.merge(disagg_merge_1, disagg_merge_2, on="region_code")
disagg_merge["value"] = disagg_merge["value_x"] + disagg_merge["value_y"]

In [ ]:
disagg_tranport = disagg_merge[["region_code", "value"]].copy()

In [ ]:
disagg_tranport["NUTS2"] = disagg_tranport["region_code"].str[:4]

In [ ]:
disagg_tranport.drop(columns="region_code", inplace=True)

In [ ]:
disagg_tranport = disagg_tranport.groupby("NUTS2").sum().reset_index()

In [ ]:
disagg_tranport_de = disagg_tranport[disagg_tranport["NUTS2"].str.startswith("DE")]
disagg_tranport_es = disagg_tranport[disagg_tranport["NUTS2"].str.startswith("ES")]

In [ ]:
edgar = pd.read_excel(
       os.path.join( "..", "..", "..", 
                    "data", 
                    "validation_data", 
                    "EDGARv8.0_total_GHG_GWP100_AR5_NUTS2_1990_2022.xlsx"),
        sheet_name="GHG by NUTS2 and Sector",
        header=0,
        index_col=0,
    )

edgar_transport = edgar[(edgar["NUTS 2"].str.startswith(("ES", "DE"))) & (edgar["Sector"] == "Transport")][["NUTS 2", "Y_2022"]].reset_index(drop=True)

In [ ]:
edgar_transport.rename(columns={"NUTS 2": "NUTS2", "Y_2022": "validation_value"}, inplace=True)
edgar_transport["validation_value"] = edgar_transport["validation_value"] /1000 # kton to Mt

In [ ]:
edgar_transport_de = edgar_transport[edgar_transport["NUTS2"].str.startswith("DE")]
edgar_transport_es = edgar_transport[edgar_transport["NUTS2"].str.startswith("ES")]

In [ ]:
set(edgar_transport_de["NUTS2"]) - set(disagg_tranport_de["NUTS2"])

In [ ]:
set(disagg_tranport_de["NUTS2"]) - set(edgar_transport_de["NUTS2"])

In [ ]:
set(edgar_transport_es["NUTS2"]) - set(disagg_tranport_es["NUTS2"])

In [ ]:
set(disagg_tranport_es["NUTS2"]) - set(edgar_transport_es["NUTS2"])

In [ ]:
diff_df = pd.merge(edgar_transport, disagg_tranport, on="NUTS2", how="outer")

In [ ]:
diff_df["perct_dev"] = ((diff_df["validation_value"] - diff_df["value"]) / diff_df["validation_value"]) * 100
diff_df["abs_perct_dev"] = abs((diff_df["validation_value"] - diff_df["value"]) / diff_df["validation_value"]) * 100

In [ ]:
diff_df = pd.merge(diff_df, nuts2, left_on="NUTS2", right_on = "code", how="outer")

In [ ]:
diff_df

In [ ]:
diff_df.loc[diff_df['NUTS2'] == "ES70", 'name'] = 'Canaries'

In [ ]:
diff_df_de = diff_df[diff_df["NUTS2"].str.startswith("DE")]
diff_df_es = diff_df[diff_df["NUTS2"].str.startswith("ES")]

In [ ]:
diff_df_de.sort_values("abs_perct_dev")

In [ ]:
len(diff_df_de[diff_df_de["abs_perct_dev"] <=20])

In [ ]:
len(diff_df_de)

In [ ]:
38-16

In [ ]:
diff_df_es.sort_values("abs_perct_dev")

In [ ]:
len(diff_df_es[diff_df_es["abs_perct_dev"] <=20])

In [ ]:
fig = plt.figure(figsize=(13, 10))
gs = fig.add_gridspec(9, 1, wspace=0.1, hspace=0)

# Germany - Deviations --------
ax1 = plt.subplot(gs[:3, :])

ax1.stem(diff_df_de['name'], diff_df_de['validation_value'], linefmt='b-', markerfmt='bo', basefmt=" ", label='EDGAR value')
ax1.stem(diff_df_de['name'], diff_df_de['value'], linefmt='r-', markerfmt='ro', basefmt=" ", label='Disaggregated value')

ax1.set_title("Germany")
ax1.set_ylabel('Emissions\n(kt CO2 equivalent)')
ax1.set_xticks([])        # Removes tick marks
ax1.set_xticklabels([])   # Removes tick labels
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc="upper right")
# Germany - Percentage deviation  --------
ax2 = plt.subplot(gs[3:4, :])

ax2.stem(diff_df_de['name'], diff_df_de['perct_dev'], linefmt='k-', markerfmt='ko', basefmt=" ", label='Percentage deviation')

ax2.set_ylabel('Percentage\ndeviation')
plt.xticks(rotation=45, ha="right") 
plt.grid(True, linestyle='--', alpha=0.6)

# Spain - Deviations --------
ax3 = plt.subplot(gs[6:8, :])

ax3.stem(diff_df_es['name'], diff_df_es['validation_value'], linefmt='b-', markerfmt='bo', basefmt=" ", label='EDGAR value')
ax3.stem(diff_df_es['name'], diff_df_es['value'],linefmt='r-', markerfmt='ro', basefmt=" ",  label='Disaggregated value')

ax3.set_title("Spain")
ax3.set_ylabel('Emissions\n(kt CO2 equivalent)')
ax3.set_xticks([])        # Removes tick marks
ax3.set_xticklabels([])   # Removes tick labels
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(loc="upper right")
# Germany - Percentage deviation  --------
ax4 = plt.subplot(gs[8:, :])

ax4.stem(diff_df_es['name'], diff_df_es['perct_dev'], linefmt='k-', markerfmt='ko', basefmt=" ", label='Percentage deviation')

ax4.set_ylabel('Percentage\ndeviation')
plt.xticks(rotation=45, ha="right") 
plt.grid(True, linestyle='--', alpha=0.6)
ax4.set_xlabel('Region name')


plt.savefig(os.path.join("..", "..",  "..", "figures", "disaggregation_validation", 
                            f"edgar_transport.png"), 
                            bbox_inches='tight')  # Save the figure as a PNG file
